In [360]:
import numpy as np 
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import sys
from dotenv import load_dotenv
load_dotenv()

True

In [361]:
token = SpotifyClientCredentials(client_id= os.getenv('SPOTIFY_CLIENT_ID'), client_secret= os.getenv('SPOTIFY_SECRET'))
sp = spotipy.Spotify(client_credentials_manager= token)

In [374]:
# Fetch songs from a single playlist
def fetch_songs(playlist_id):
    playlist_features_list = ["artist", "album", "track_name", "track_id"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    results = sp.playlist_tracks(playlist_id)
    tracks_data = results['tracks']
    tracks = []

    # Collect every page of track data
    print('started reading tracks from playlist')
    while tracks_data['next']:
        tracks_data = sp.next(tracks_data)
        tracks.extend(tracks_data['items'])
    print('finished reading tracks')
    for track in tracks:
        playlist_features = {}
        
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        playlist_features["popularity"] = track["track"]["popularity"]

        # Concat
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
    ids = playlist_df['track_id'].to_list()
    # Grab all the song analysis data
    analysis_features = ["acousticness", "liveness", "loudness", "speechiness", "tempo", 
                         "time_signature", "tempo", "energy", "instrumentalness", "time_signature", "id"]
    analysis_df = pd.DataFrame()
    analysis_data = {}
    print('started reading features')
    for i in range(len(ids)):
        song_analysis = sp.audio_features(ids[i])
        for j in range(len(analysis_features)):
            analysis_data[analysis_features[j]] = song_analysis[0][analysis_features[j]]
        curr_anal_df = pd.DataFrame(analysis_data, index = [0])
        analysis_df = pd.concat([analysis_df, curr_anal_df], ignore_index = True)
    print('finished reading features')

    # TODO: Remove
    tracks_df = pd.merge(left=playlist_df, right=analysis_df, left_on='track_id', right_on='id')

    return tracks_df 
    


In [375]:
fetch_songs('4rnleEAOdmFAbRcNCgZMpY?si=4c088aeade4b43e5')

started reading tracks from playlist
finished reading tracks
started reading features
finished reading features


,artist,album,track_name,track_id,popularity,acousticness,liveness,loudness,speechiness,tempo,time_signature,energy,instrumentalness,id
0,Leona Lewis,Spirit,Bleeding Love,7wZUrN8oemZfsEd1CGkbXE,78.0,0.18800,0.1460,-5.886,0.0357,104.036,4,0.656,0.000000,7wZUrN8oemZfsEd1CGkbXE
1,Kelis,Tasty,Milkshake,2cMTIlktg3M9mXYqCPqw1J,70.0,0.00986,0.2060,-6.068,0.0439,112.968,4,0.774,0.034600,2cMTIlktg3M9mXYqCPqw1J
2,Kelly Clarkson,Greatest Hits - Chapter One,Stronger (What Doesn't Kill You),35a4Kzyo9XY4FI9FaSIzzF,47.0,0.04110,0.1120,-5.908,0.0482,116.012,4,0.923,0.000000,35a4Kzyo9XY4FI9FaSIzzF
3,Vanessa Hudgens,V,Say OK,5cfPxiul0cdhPYD2uzuQSg,0.0,0.00673,0.0884,-7.176,0.0468,125.026,3,0.696,0.000955,5cfPxiul0cdhPYD2uzuQSg
4,Emily Osment,Fight Or Flight,Lovesick,2w55BLEMG97Smym2V6itnx,0.0,0.00414,0.3530,-5.621,0.0604,129.880,3,0.715,0.000000,2w55BLEMG97Smym2V6itnx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5200,MARINA,The Family Jewels,Hermit the Frog,4Zcz6saEkOII3PlXd9gN3o,73.0,0.24300,0.1990,-4.545,0.0312,122.034,4,0.679,0.000000,4Zcz6saEkOII3PlXd9gN3o
5201,Olivia Rodrigo,deja vu,deja vu,61KpQadow081I2AsbeLcsb,30.0,0.59300,0.3410,-7.236,0.1160,181.088,4,0.610,0.000011,61KpQadow081I2AsbeLcsb
5202,BIA,FOR CERTAIN,WHOLE LOTTA MONEY,5yorXJWdBan1Vlh116ZtQ7,72.0,0.09040,0.3250,-5.019,0.3680,81.008,4,0.371,0.000000,5yorXJWdBan1Vlh116ZtQ7
5203,Ashnikko,DEMIDEVIL,Slumber Party (feat. Princess Nokia),11ZulcYY4lowvcQm4oe3VJ,80.0,0.00151,0.1010,-8.981,0.0795,105.012,4,0.398,0.000039,11ZulcYY4lowvcQm4oe3VJ


In [ ]:


#Function skeletons

def groupby_genre(songs_df):
    pass
def groupby_artist(songs_df):
    pass
def groupby_popularity(songs_df):
    pass
def groupby_audio_features(songs_df):
    pass